Scrape Questions and metadata

In [4]:
import sys
sys.path.insert(0, "../../") 
from src.GetQuestionsList import GetQuestionsList
from src.GetQuestionInfo import GetQuestionInfo
from src.utils import combine_list_and_info, get_all_questions_body
import pandas as pd
from tqdm import tqdm
import pickle
import numpy as np

Query the question list

In [2]:
ls = GetQuestionsList()
ls.scrape()
ls.to_csv("../data/")

Note: The default ALL_JSON_URL might be out-of-date. Please update it by going to https://leetcode.com/problemset/all/ and exploring the Networks tab for a query returning all.json.
Scraping companies ... Done
Scraping questions list ... Done
Extracting question topics ... Done
Scraping Categories ... Done
Scraping Topic Tags ... Done
Extracting question category ... Done


Query Individual question info such as the body, test cases, constraints, hints, code stubs, etc.

In [3]:
# questions_info = pd.read_csv("../data/questions.csv")
# questions_info_list = get_all_questions_body(
#     questions_info["titleSlug"].tolist(),
#     questions_info["paidOnly"].tolist(),
#     filename="../data/questionBody.pickle",
# )
# # questions_info_list = []
# # for i, (titleSlug, paidOnly) in enumerate(tqdm(questions_info[["titleSlug", "paidOnly"]].values)):
# #     if not paidOnly:
# #         questions_info_list.append(GetQuestionInfo(titleSlug).scrape())
# #     if i % 10 == 0:
# #         with open("./dump.pickle", "wb") as f:
# #             pickle.dump(questions_info_list, f)

# # with open("./dump.pickle", "wb") as f:
# #     pickle.dump(questions_info_list, f)


""" Run the above code stub once and save the data as a pickle file. Using this data
from now on since the above code stub is time consuming. """

with open("../data/questionBody.pickle", "rb") as f:
    data = pickle.load(f)
questions_body = pd.DataFrame(data).drop(columns=["titleSlug"])
questions_body["QID"] = questions_body["QID"].astype(int)


Create a new dataframe with all the questions with their metadata and body information.

In [4]:
questions = combine_list_and_info(info_df = questions_body, list_df=ls.questions)

Data Upload to SUPABASE table|s

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from dotenv import dotenv_values

config = dotenv_values("../.env")


In [22]:
engine = create_engine(
    f"postgresql://{config['SUPABASE_USERNAME']}:{config['SUPABASE_PASSWORD']}@{config['SUPABASE_HOSTNAME']}:{config['SUPABASE_PORT']}/{config['SUPABASE_DBNAME']}",
    echo=True,
)
questions.to_sql(con=engine, name="questions", if_exists="append", index=False)
ls.topicTags.to_sql(con=engine, name="topic_tags", if_exists="append", index=False)
ls.categories.to_sql(con=engine, name="categories", if_exists="append", index=False)
ls.companies.to_sql(con=engine, name="companies", if_exists="append", index=False)
ls.questionTopics.to_sql(
    con=engine, name="question_topics", if_exists="append", index=True, index_label="id"
)
ls.questionCategory.to_sql(
    con=engine,
    name="question_category",
    if_exists="append",
    index=True,
    index_label="id",
)


Upload solutions to questions

In [2]:
import sys
sys.path.insert(0, "../../") 

from sqlalchemy.engine import result
import sqlalchemy
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    Numeric,
    Integer,
    VARCHAR,
    update,
)
import glob
from dotenv import dotenv_values
from src.utils import extract_solutions



In [3]:

config = dotenv_values("../../.env")
eng = create_engine(
    f"postgresql://{config['SUPABASE_USERNAME']}:{config['SUPABASE_PASSWORD']}@{config['SUPABASE_HOSTNAME']}:{config['SUPABASE_PORT']}/{config['SUPABASE_DBNAME']}",
    echo=True,
)

In [6]:
filesWithSolutions = glob.glob("../../src/questions/q_*.py")
for file in filesWithSolutions:
    extract_solutions(filesWithSolutions[2], engine=eng,  upload= True)

2023-01-16 19:25:44,434 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-01-16 19:25:44,435 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-01-16 19:25:44,452 INFO sqlalchemy.engine.Engine select current_schema()
2023-01-16 19:25:44,453 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-01-16 19:25:44,471 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-01-16 19:25:44,471 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-01-16 19:25:44,494 INFO sqlalchemy.engine.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind in ('r', 'p')
2023-01-16 19:25:44,497 INFO sqlalchemy.engine.Engine [generated in 0.00295s] {'schema': 'public'}
2023-01-16 19:25:44,522 INFO sqlalchemy.engine.Engine 
            SELECT c.oid
            FROM pg_catalog.pg_class c
            LEFT JOIN pg_catalog.pg_namespace n ON n.oid = c.relnamespace
            WHERE (pg_catalog.pg_table_is_visible(c.oid))
            